# LeNet-5 в Keras

В настоящей тетради мы построим глубокую свёрточную нейронную сеть, которая будет классифицировать цифры из набора данных MNIST. Основой для данной сети станет небезызвестная LeNet-5.

***

## Зависимости

Для реализации архитектуры сети нам понадобятся библиотеки, импортированные ниже.

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D

Приступим теперь к загрузке и обработке данных.

***

## Данные

В качестве данных для нашего улучшеного клона LeNet-5 был выбран огромный набор цифр, написанных от руки. Загрузим датасет и выполним необходимые шаги предварительной обработки.

In [2]:
(x_train, y_train), (x_valid, y_valid) = mnist.load_data()

x_train, x_valid = (
    x_train.reshape(len(x_train), 28, 28, 1).astype("float32"), 
    x_valid.reshape(len(x_valid), 28, 28, 1).astype("float32"),
)
x_train /= 255
x_valid /= 255

n_classes = 10
y_train = keras.utils.to_categorical(y_train, n_classes)
y_valid = keras.utils.to_categorical(y_valid, n_classes)

Здесь мы обязательно меняем форму массивов, чтоб обозначить количество каналов, затем нормализуем данные и приводим значения классов к категориальному формату.

***

## Архитектура Сети

Теперь настало время определить архитектуру сети.

Обозначим каждый слой и его значения:
1. Свёрточный слой с размером ядра 3 без дополнения, шагом 1 и функцией активации ReLU;
2. Пулинговый слой по максимуму с размером 2 на 2;
3. Слой прореживания (исключения избыточных нейронов);
4. Слой понижения размерности для совместимости с полносвязным слоем;
5. Полносвязный слой со 128 нейронами и функцией активации ReLU;
6. Очередной слой прореживания;
7. Полносвязный слой с 10 нейронами функцией активации Softmax.

Ниже представлен код инициализации и таблица для вышеописанной модели.

In [3]:
model = Sequential(
    [
        Input(shape=(28, 28, 1)),
        Conv2D(32, kernel_size=(3, 3), activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation="relu"),
        Dropout(0.5),
        Dense(n_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 5408)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         692,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 693,962 (2.65 MB)

 Trainable params: 693,962 (2.65 MB)

 Non-trainable params: 0 (0.00 B)

Скопилируем модель со следующими гиперпараметрами:
* функция потерь — категориальная перекрёстная энтропия,
* оптимимзатор — Adam,
* метрики — точность.

Код проще некуда. Его можно наблюдать ниже.

In [4]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

Теперь настало время обучения.

***

## Обучение Сети

Обучим получившуюся нейронную сеть. Прогонять будем в течение десяти эпох по 128 изображений за партию.

In [5]:
model.fit(x_train, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(x_valid, y_valid))

Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.8082 - loss: 0.6178 - val_accuracy: 0.9682 - val_loss: 0.1081
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9520 - loss: 0.1661 - val_accuracy: 0.9784 - val_loss: 0.0684
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9651 - loss: 0.1154 - val_accuracy: 0.9808 - val_loss: 0.0525
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9717 - loss: 0.0958 - val_accuracy: 0.9817 - val_loss: 0.0497
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9757 - loss: 0.0788 - val_accuracy: 0.9845 - val_loss: 0.0415
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9779 - loss: 0.0703 - val_accuracy: 0.9850 - val_loss: 0.0412
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9810 - loss: 0.0647 - val_accuracy: 0.9864 - val_loss: 0.0396
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9821 - loss: 0.0580 - val_accuracy: 0.

Как видим, наша модель теперь может угадывать рукописные цифры с вероятностью почти 0.99.